In [ ]:
### Extractive text summarization
This notebook has a function at the end that can be used on any 

In [2]:
from tqdm import tqdm
import joblib
data = joblib.load('final_df3.pkl')

In [3]:
def wrap(mytext):
  import textwrap
  wrapper = textwrap.TextWrapper(width=60)
  word_list = wrapper.wrap(text=mytext) 

  # Print each line. 
  for element in word_list: 
      print(element) 


In [6]:
print(data.columns)
textonly = list(data.text)

Index(['title', 'summary_x', 'URL', 'published', 'keywords', 'summary_y',
       'text'],
      dtype='object')


In [4]:
# Main function generate_summary defined in this cell, usage is generate_summary( text, number_of_sentences_to_return)
# https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70

from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import nltk
nltk.download('stopwords')
import textwrap
 
def read_article(text):
  # file = open(file_name, 'r')
  # filedata = file.readlines()
  # filedata = [x for x in filedata if x != '\n']
  filedata = text
  filedata = [x.replace('\n',' ') for x in filedata]
  filedata = ''.join(filedata)
  filedata = filedata.split('. ')
  sentences = []

  for sentence in filedata:
    sentences.append(sentence.replace('[^a-zA-Z]', ' ').split(' '))
     
  return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)
    if len(sentences) < top_n:
      top_n = len(sentences)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Output the summarized text
    summary = ". ".join(summarize_text)
    # print("Summarize Text: \n", ". ".join(summarize_text))
    return summary

    # wrapper = textwrap.TextWrapper(width=60)
    # word_list = wrapper.wrap(text=summary) 
  
    # # Print each line. 
    # for element in word_list: 
    #     print(element) 

    # # print(type(summary))
    # # print(summary)
    

# let's begin
# generate_summary( "msft.txt", 2)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
summ = []
err = []
for x in tqdm(textonly):
  try:
    summ.append(generate_summary(x, 5))
  except:
    err.append(textonly.index(x))
    summ.append('Could not generate')
    

    


 20%|█▉        | 794/3985 [02:12<19:11,  2.77it/s]/usr/local/lib/python3.6/dist-packages/nltk/cluster/util.py:133: RuntimeWarning: invalid value encountered in true_divide
  sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))
100%|██████████| 3985/3985 [12:35<00:00,  5.27it/s]


In [8]:
x = textonly[345]
textonly[345]

'John J. Mooney, an inventor of the catalytic converter, the small and ubiquitous device that makes the engines that power everything from cars to lawn mowers less polluting and more fuel efficient, died on June 16 at his home in Wyckoff, N.J. He was 90.\n\nThe cause was complications of a stroke, his daughter Elizabeth Mooney Convery said.\n\nMr. Mooney was a high school graduate working as a clerk at a gas company when his colleagues encouraged him to pursue a college education. After earning a bachelor’s degree and two master’s degrees, he went on to receive 17 patents during his 43-year career with the Englehard Corporation in Iselin, N.J. (now the Catalyst Division of the German chemical manufacturer BASF).\n\nAmong them was the three-way catalytic converter, which has been described by the Society of Automotive Engineers as among the 10 most important innovations in the history of the automobile.\n\nThe Environmental Protection Agency has estimated that tailpipe emissions from th

In [14]:
len(textonly[:3])

3

In [15]:
file = open('input.txt','w')
for element in textonly[:4]:
    file.write(element+'\n')
file.close()

In [11]:
generate_summary(x, 4)

' Mr.  The cause was complications of a stroke, his daughter Elizabeth Mooney Convery said.  Among them was the three-way catalytic converter, which has been described by the Society of Automotive Engineers as among the 10 most important innovations in the history of the automobile.  The Environmental Protection Agency has estimated that tailpipe emissions from the newest passenger cars, sport utility vehicles, trucks and buses generate about 99 percent less smog-producing exhaust and soot than those from the 1970 models did.'

In [ ]:
err

[795, 858, 1405, 1725, 1893, 1998, 2100, 2332, 2594, 3018, 3132, 3446, 3783]

In [ ]:
data.iloc[795].URL

'https://www.kaspersky.com/blog/raising-digital-parents/35633/'

In [ ]:
i=3445
print(wrap(summ[i]))
print('\n-------------')
print(wrap(summ[i]))

 Nines is the first company to receive FDA clearance for use
of an AI tool in triaging intracranial hemorrhage and mass
effect.  The NinesAI tool is being made available free to
Nines customers who use its teleradiology product, and also
to any existing customers for use in-house at radiology
practices with no additional costs.  NinesAI is designed to
identify possible emergent cases of intracranial hemorrhage
and mass effect conditions in patients, helping radiologists
prioritize cases to review for further study. Medical
startup Nines, which has developed an AI-based triage tool
that has received clearance from the U.S. Food and Drug
Administration (FDA) for use, is making that tool available
for free to all until June 30 to help address the growing
burden on radiology diagnostics departments as COVID-19
continues to reshape the healthcare landscape in the U.S
None

-------------
 Nines is the first company to receive FDA clearance for use
of an AI tool in triaging intracranial hemor

In [ ]:
wrap(textonly[796])

To hear more audio stories from publishers like The New York
Times, download Audm for iPhone or Android.  A virus, at
heart, is information, a packet of data that benefits from
being shared.  The information at stake is genetic:
instructions to make more virus. Unlike a truly living
organism, a virus cannot replicate on its own; it cannot
move, grow, persist or perpetuate. It needs a host. The
viral code breaks into a living cell, hijacks the genetic
machinery and instructs it to produce new code — new virus.
President Trump has characterized the response to the
pandemic as a “medical war,” and described the virus behind
it as, by turns, “genius,” a “hidden enemy” and “a monster.”
It would be more accurate to say that we find ourselves at
odds with a microscopic photocopy machine. Not even that: an
assembly manual for a photocopier, model SARS-CoV-2.  For at
least six months now, the virus has replicated among us. The
toll has been devastating. Officially, more than six million
people 

In [ ]:
wrap(summ[2])

 [...]  The new SGX attacks are known as SGAxe and
CrossTalk. Here's a new vulnerability:  On Tuesday, two
separate academic teams disclosed two new and distinctive
exploits that pierce Intel's Software Guard eXtension, by
far the most sensitive region of the company's processors.
Remember Spectre and Meltdown? Back in early 2018, I wrote:
Spectre and Meltdown are pretty catastrophic
vulnerabilities, but they only affect the confidentiality of
data.  That has turned out to be true. These will be
similarly impossible to fix, and the only strategy will be
to throw our devices away and buy new ones


In [ ]:
data[1]

'This is a weird story:\n\nHernandez was able to evade capture for so long because he used Tails, a version of Linux designed for users at high risk of surveillance and which routes all inbound and outbound connections through the open-source Tor network to anonymize it. According to Vice, the FBI had tried to hack into Hernandez\'s computer but failed, as the approach they used "was not tailored for Tails." Hernandez then proceeded to mock the FBI in subsequent messages, two Facebook employees told Vice.\n\nFacebook had tasked a dedicated employee to unmasking Hernandez, developed an automated system to flag recently created accounts that messaged minors, and made catching Hernandez a priority for its security teams, according to Vice. They also paid a third party contractor "six figures" to help develop a zero-day exploit in Tails: a bug in its video player that enabled them to retrieve the real I.P. address of a person viewing a clip. Three sources told Vice that an intermediary pas

In [ ]:
read_article(textonly)

Windows 7 users should be aware by now that while a fair
number of flaws addressed this month by Microsoft affect
Windows 7 systems, this operating system is no longer being
supported with security updates (unless you’re an enterprise
taking advantage of Microsoft’s paid extended security
updates program, which is available to Windows 7
Professional and Windows 7 enterprise users). Perhaps most
troubling of these (CVE-2020-1301) is a remote code
execution bug in SMB capabilities built into Windows 7 and
Windows Server 2008 systems — both operating systems that
Microsoft stopped supporting with security updates in
January 2020. Microsoft Office and Excel get several updates
this month. Microsoft today released software patches to
plug at least 129 security holes in its Windows operating
systems and supported software, by some accounts a record
number of fixes in one go for the software giant. Adobe also
released security updates for its Experience Manager and
Framemaker products


In [ ]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article('textfile.txt')

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

In [ ]:
len(read_article(textonly[0]))

4